In [ ]:
pip install --upgrade datasets

In [ ]:
import pandas as pd
from datasets import Dataset
from huggingface_hub.utils import get_session
import string
from nltk.corpus import stopwords


ImportError: cannot import name 'get_session' from 'huggingface_hub.utils' (/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/__init__.py)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("imdb")

ImportError: cannot import name 'get_session' from 'huggingface_hub.utils' (/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/__init__.py)

In [ ]:
ds = load_dataset('imdb')

In [ ]:
ds

In [ ]:
ds['train'].features

In [ ]:
ds_train = pd.DataFrame(ds['train'])
ds_test = pd.DataFrame(ds['test'])

In [ ]:
ds_train.head()

In [ ]:
#Removing punctuation from the reviews

In [ ]:
string.punctuation

In [ ]:
#Create function to remove punctuation
def remove_punctuation(text):
  text = "".join([char for char in text if char not in string.punctuation])

  return text

In [ ]:
ds_train['No Punct Review'] = ds_train['text'].apply(lambda x: remove_punctuation(x))
ds_test['No Punct Review'] = ds_test['text'].apply(lambda x: remove_punctuation(x))


In [ ]:
ds_train.sample(10)

In [ ]:
#Tokenizing each review

def lowercase_tokenizer(text):
  token = text.lower().split()

  return token

In [ ]:
#Creating a new column for tokens
ds_train['tokens'] = ds_train['No Punct Review'].apply(lambda x: lowercase_tokenizer(x))
ds_test['tokens'] = ds_test['No Punct Review'].apply(lambda x: lowercase_tokenizer(x))

In [ ]:
ds_train.head()

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
stop_words = stopwords.words('english')

In [ ]:
#Removing stop words
def remove_stopwords(text):
  text = [word for word in text if word not in stop_words]

  return text

In [ ]:
#Creating a new column for token with no stop words
ds_train['no_stopword_tokens'] = ds_train['tokens'].apply(lambda x: remove_stopwords(x))
ds_test['no_stopword_tokens'] = ds_test['tokens'].apply(lambda x: remove_stopwords(x))

In [ ]:
ds_train.head()

In [ ]:
ds_test.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

n_words = 1000

#Vectorizing each word using CountVectorizer
vectorizer = CountVectorizer(max_features=n_words, stop_words='english')

vectorizer.fit(ds_train['text'])

In [ ]:
print('Vocabulary:', vectorizer.vocabulary_)

In [ ]:
from gensim.models import Word2Vec

tokenized_train_text = ds_train['no_stopword_tokens']
tokenized_test_text = ds_test['no_stopword_tokens']

#Using Word2Vec to embed the words
word2vec_train = Word2Vec(sentences=tokenized_train_text, min_count=2)
word2vec_test = Word2Vec(sentences=tokenized_test_text, min_count=2)

word_vectors_train = word2vec_train.wv
word_vectors_test = word2vec_test.wv

In [ ]:
word_vectors_train.most_similar("mad")

In [ ]:
word_vectors_train.similarity("great","amazing")

In [ ]:
from datasets import Dataset, DatasetDict
# assign the splits
train = Dataset.from_pandas(ds_train)
test = Dataset.from_pandas(ds_test)
# reconstruct both datasets into a Dataset Dict object
new_ds = DatasetDict(
    {
        'train': train,
        'test': test
    }
)
# view the resulting dataset dict object
new_ds